# How to crawl data from Twitter without using Twitter API
* Connect to the link
* Parser html
* Store data

In [38]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [40]:
company_list = ['ClevelandClinic']  #keyword
startID_list = ['914710283767005184'] #twitter ID  https://twitter.com/MikeLevinCA/status/913394885352349704

** Connect to the link **
1. Use Developer Tools (right click of 'Inspect' in Chrome). 
2. Click 'Network' to find the real link which includes the information you need. Usually it is different from the website in your browser. In this case, the real link starts with 'timeline?'.
3. Click 'Headers' to find 'Request Headers' and 'Query String Paramters', which will be needed when you set header and param.
4. Copy all the information in 'Request Headers' to header={}, except 'cookie'. Note that 'user-agent' is the most import element in this method. 
5. Copy all the information in 'Query String Paramters' to param{}.
6. Set response.


In [41]:
# First test with neurodiversity microsoft since:2015-04-30 until:2017-04-30
# https://twitter.com/search?l=en&q=neurodiversity%20microsoft%20%20since%3A2015-01-01%20until%3A2017-05-31&src=typd&lang=en
def connectTwitter(company,endID,startID):
    url = 'http://twitter.com/search'
    param = {
        'vertical':'default',
        'q':'since:2017-06-01 until:2017-09-28 '+company,# since:2015-01-01 until:2017-05-31 #time
        'l':'en',
        'src':'typd',
        'include_available_features':'1',
        'include_entities':'1',
        'lang':'en',
        'max_position':'TWEET-'+endID+'-'+startID+'-BD1UO2FFu9QAAAAAAAAETAAAAAcAAAASAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA',
        'reset_error_state':'false'
    } # TWEET-857-857 means starting with the first page.
    header = {
        'accept':'application/json, text/javascript, */*; q=0.01',
        'accept-encoding':'gzip, deflate, sdch, br',
        'accept-language':'en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4',
        'dnt':'1',
        'referer':'https://twitter.com/search?l=&q=&src=typd&vertical=&max_position=&',# Remove the information that
                                                                                       # param has included.
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
        'x-requested-with':'XMLHttpRequest',
        'x-twitter-active-user':'yes'
    }
    response = requests.get(url,params=param, headers=header)
    html = response.content.decode('utf-8')
    soup = BeautifulSoup(html,'lxml')
    
    try:
        tweets_html_id = soup.find_all('div',class_='stream')[0].find_all('li')
        tweets_id = [tweet_html_id.get('data-item-id') for tweet_html_id in tweets_html_id]
        tweets_id = [t for t in tweets_id if t is not None]
        next_endID = tweets_id[len(tweets_id)-1]
        
    except IndexError:
        soup = None
        next_endID = None

    return soup,next_endID

** Parser HTML and Store data to DataFrame**


In [42]:
# DOM Parser
def parser_DF(soup):
    tweets_html = soup.find_all('li',class_='js-stream-item stream-item stream-item ') # note of the space
    # get user name
    tweets_username = [tweet_html.find('div').get('data-screen-name') 
                           for tweet_html in tweets_html]
    # get DataTime
    tweets_datetime = [tweet_html.find('small',class_='time').find('a').get('title') 
                           for tweet_html in tweets_html]
    # get content
    tweets_content = [tweet_html.find('p',class_='TweetTextSize js-tweet-text tweet-text').text.strip() 
                          for tweet_html in tweets_html]
    # get replies, retweets, likes
    tweets_replies = [tweet_html.find('span',class_='ProfileTweet-action--reply u-hiddenVisually').find('span')
                          .get('data-tweet-stat-count') for tweet_html in tweets_html]
    tweets_retweet = [tweet_html.find('span',class_='ProfileTweet-action--retweet u-hiddenVisually').find('span')
                          .get('data-tweet-stat-count') for tweet_html in tweets_html]
    tweets_likes = [tweet_html.find('span',class_='ProfileTweet-action--favorite u-hiddenVisually').find('span')
                          .get('data-tweet-stat-count') for tweet_html in tweets_html]
    # get image link
    tweets_image=[]
    tweets_image_html = [tweet_html.find('div',class_='AdaptiveMedia-singlePhoto') for tweet_html in tweets_html]
    for tweet_image_html in tweets_image_html:
        if tweet_image_html is not None:
            tweets_image.append(tweet_image_html.find('div').get('data-image-url'))
        else:
            tweets_image.append(None)
    # get verified account
    tweets_verified=[]
    tweets_html[6].find('span',class_='Icon Icon--verified')
    tweets_verified_html = [tweet_html.find('span',class_='Icon Icon--verified') for tweet_html in tweets_html]
    for tweet_verified_html in tweets_verified_html:
        if tweet_verified_html is None:
            tweets_verified.append(0)
        else:
            tweets_verified.append(1)
    
    # store data to dataframe        
    tweets = [('username',tweets_username),
              ('verified',tweets_verified),
              ('DateTime',tweets_datetime),
              ('content',tweets_content),
              ('likes',tweets_likes),
              ('retweet',tweets_retweet),
              ('replies',tweets_replies),
              ('image',tweets_image)]

    tweets_df = pd.DataFrame.from_items(tweets)
    
    return tweets_df

** Call Method **

In [43]:
# Automatically get lastID and go to next page
companies_html = [] # global variable
for i in range(len(company_list)): # for loop of company_list
    soup_list=[]
    id_list=[]

    if len(id_list)==0: # First page: startID and endID would be the same
        s = connectTwitter(company_list[i],startID_list[i],startID_list[i])
        soup_list.append(s[0])
        id_list.append(s[1])

    if len(id_list)>=1:
        while(id_list[-1] is not None):
            s = connectTwitter(company_list[i],id_list[-1],startID_list[i])
            soup_list.append(s[0])
            id_list.append(s[1])

    if(id_list[-1] is None): # because of the try catch in the def
        del soup_list[-1]
        del id_list[-1]
        
    companies_html.append(soup_list)

KeyboardInterrupt: 

In [ ]:
# parse html and merge dataframe ---CSV
companies_df_list = [] # global variable
for j in range(len(companies_html)):
    tweets_df_list = []
    for soup in companies_html[j]:
        tweets_df = parser_DF(soup)
        tweets_df_list.append(tweets_df)
    companies_df_list.append(pd.concat(tweets_df_list,ignore_index=True))
    #companies_df_list[j].to_csv(company_list[j]+'.csv')

In [25]:
companies_df_list[0]

IndexError: list index out of range

In [17]:
companies_df_list[1]

IndexError: list index out of range